# 🐿 나의 작은 생태계

---
## 개요
환경빅데이터플랫폼 혁신서비스 아이디어 공모전 ( 2021.01.27 ~ 02.17 ) 

* 스마트폰 사용 시간이 점점 늘어가는 아이들을 위해 야외활동을 유발하는 서비스

     * 우리집 주변의 동물, 조류, 곤충 등의 서식지와 정보를 알려줌
     
* 팀원 : `허정훈`,  `구원혁`, `민성훈`, `진효정`

* 필요 라이브러리 설치

In [13]:
!conda install -c conda-forge pyproj
!conda install -c conda-forge haversine
!pip install wikipedia-API

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



You should consider upgrading via the 'c:\users\82103\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import folium
import pandas as pd
from pyproj import Proj, transform
from haversine import haversine
import decimal
import warnings
warnings.filterwarnings('ignore')

## 데이터
* 데이터 출처 :  환경빅데이터플랫폼 [link]((https://www.bigdata-environment.kr/user/data_market/list.do))
* 50만개 정도의 곤충, 포유류, 조류 등의 동물 데이터가 있습니다.
* 그 종의 종명, 학명, 서식지 좌표 등의 데이터로 이루어져 있습니다.

In [2]:
df_ins = pd.read_csv('./data/202011_자연환경조사_곤충_점.csv')
df_rep = pd.read_csv('./data/202011_자연환경조사_양서파충류_점.csv')
df_spi = pd.read_csv('./data/202011_자연환경조사_저서무척추동물_점.csv')
df_bird = pd.read_csv('./data/202011_자연환경조사_조류_점.csv')
df_mam = pd.read_csv('./data/202011_자연환경조사_포유류_점.csv')

In [3]:
df_ins = df_ins[['spce_id','examin_realm_se_code_nm','spcs_lcnm','spcs_scncenm','geom']].copy()
df_rep = df_rep[['spce_id','examin_realm_se_code_nm','spcs_lcnm','spcs_scncenm','geom']].copy()
df_spi = df_spi[['spce_id','examin_realm_se_code_nm','spcs_lcnm','spcs_scncenm','geom']].copy()
df_bird = df_bird[['spce_id','examin_realm_se_code_nm','spcs_lcnm','spcs_scncenm','geom']].copy()
df_mam = df_mam[['spce_id','examin_realm_se_code_nm','spcs_lcnm','spcs_scncenm','geom']].copy()

In [11]:
df_mam.head()

,spce_id,examin_realm_se_code_nm,spcs_lcnm,spcs_scncenm,geom
0,9000000000549745,포유류,너구리,"Nyctereutes procyonoides (Gray, 1834)",POINT (149858.01580000017 495400.6850000005)
1,9000000000549747,포유류,고라니,"Hydropotes inermis Swinhoe, 1870",POINT (149734.41930000018 495408.1886999998)
2,9000000000549748,포유류,고라니,"Hydropotes inermis Swinhoe, 1870",POINT (149494.92399999965 495431.14650000073)
3,9000000000549750,포유류,멧돼지,"Sus scrofa Linnaeus, 1758",POINT (217427.9857999999 263351.8610999994)
4,9000000000549752,포유류,고라니,"Hydropotes inermis Swinhoe, 1870",POINT (220471.97030000016 263481.4341000002)


* 우리가 흔히 쓰는 경위도가 형태가 아니기 때문에 우리가 쓰기 좋은 형태로 전처리

In [14]:
df_ins['lati'] = df_ins['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_ins['long'] = df_ins['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_rep['lati'] = df_rep['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_rep['long'] = df_rep['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_spi['lati'] = df_spi['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_spi['long'] = df_spi['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_bird['lati'] = df_bird['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_bird['long'] = df_bird['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_mam['lati'] = df_mam['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_mam['long'] = df_mam['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

In [15]:
df_ins.head(5)

,spce_id,examin_realm_se_code_nm,spcs_lcnm,spcs_scncenm,geom,lati,long
0,5000000001325505,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",POINT (141724.9611999998 427724.3739),427724.3739,141724.9611999998
1,5000000001493544,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",POINT (159314.02529999986 94114.0251000002),94114.0251000002,159314.02529999986
2,5000000001335750,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",POINT (234776.7951999996 344101.1821999997),344101.1821999997,234776.7951999996
3,5000000001464084,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",POINT (182054.65479999967 354509.0923999995),354509.0923999995,182054.65479999967
4,5000000001365572,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",POINT (228545.61799999978 349336.3881999999),349336.3881999999,228545.61799999978


* 곤충~동물까지 모두 데이터 병합

In [16]:
df_all = pd.concat([df_ins, df_rep, df_spi, df_bird, df_mam]).copy()
df_all = df_all[['spce_id','examin_realm_se_code_nm','spcs_lcnm','spcs_scncenm','lati','long',]].reset_index(drop=True)

## 좌표 변환

* 국토지리정보원 표준 좌표계(중부원점 EPSG : 5186)인 좌표계를 GPS가 사용하는 좌표계인 EPSG : 4236로 변환 시켜 줌 - PyProj 라이브러리 사용

In [17]:
inProj = Proj(init='epsg:5186')
outProj = Proj(init='epsg:4326')

kr_long, kr_lati = transform(inProj,outProj, list(df_all['lati']), list(df_all['long']))

df_all['kr_lati'] = kr_lati
df_all['kr_long'] = kr_long

In [18]:
df_all.head()

,spce_id,examin_realm_se_code_nm,spcs_lcnm,spcs_scncenm,lati,long,kr_lati,kr_long
0,5000000001325505,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",427724.3739,141724.9611999998,33.845273,129.460214
1,5000000001493544,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",94114.0251000002,159314.02529999986,34.023060,125.853578
2,5000000001335750,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",344101.1821999997,234776.7951999996,34.698533,128.572714
3,5000000001464084,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",354509.0923999995,182054.65479999967,34.221949,128.676737
4,5000000001365572,곤충,좀남색잎벌레,"Gastrophysa atrocyanea Motschulsky, 1860",349336.3881999999,228545.61799999978,34.641635,128.628731


## 유저 정보 입력

* 유저가 원하는 경위도와 반경 입력
    * 부산대학교와 3km 이내의 동물들을 찾아본다 - haversine 라이브러리 사용

In [6]:
#  user = (35.233512, 129.0722714)
user = list(map(decimal.Decimal, input().split(',')))
# 유저의 경위도에서 0.5도 이상씩 벌어지는 데이터는 제외
df_want = df_all[(user[0]-decimal.Decimal(0.5) < df_all['kr_lati']) & (user[0]+decimal.Decimal(0.5) > df_all['kr_lati'])& (user[1]+decimal.Decimal(0.5) > df_all['kr_long'])& (
    user[1]+decimal.Decimal(0.5) > df_all['kr_long'])].reset_index(drop=True)

want_list = []
for i in df_want.index:
    if haversine(user, (df_want['kr_lati'][i], df_want['kr_long'][i]), unit = 'km') <= 3:
        want_list.append(i)
        
df_find = df_want.loc[want_list].copy()

35.233512, 129.0722714


In [20]:
df_find = df_find[['spce_id','examin_realm_se_code_nm','spcs_lcnm','spcs_scncenm','kr_lati','kr_long',]]

In [21]:
df_find.head()

,spce_id,examin_realm_se_code_nm,spcs_lcnm,spcs_scncenm,kr_lati,kr_long
565,5000000001349650,곤충,어리별쌍살벌,"Polistes mandarinus Saussure, 1853",35.244446,129.049596
614,5000000001349610,곤충,별쌍살벌,"Polistes snelleni Saussure, 1862",35.244446,129.049596
619,5000000001349646,곤충,별쌍살벌,"Polistes snelleni Saussure, 1862",35.244446,129.049596
761,5000000001349645,곤충,민호리병벌,"Eumenes rubronotatus P?ez, 1905",35.244446,129.049596
884,5000000001347826,곤충,알락하늘소,"Anoplophora chinensis (F?ster), 1771",35.252430,129.083013


## 시각화
* 곤충 : 노란색,
  양서파충류 : 파란색,
  저서무척추동물 : 빨간색,
  조류 : 자주색,
  포유류 : 초록색
    * 원하는 마커를 클릭하면, 종과 이름, 사진 시각화 -> 더 알아보기 클릭 시 해당동물의 위키피디아 정보 제공

In [9]:
import wikipediaapi
import requests
import json

# Json으로 부터 이미지 파일 불러오는 함수
def get_wiki_main_image(title):
    url = 'https://ko.wikipedia.org/w/api.php'
    data = {
        'action' :'query',
        'format' : 'json',
        'formatversion' : 2,
        'prop' : 'pageimages|pageterms',
        'piprop' : 'original',
        'titles' : title
    }
    response = requests.get(url, data, verify=False)
    json_data = json.loads(response.text)
    return json_data['query']['pages'][0]['original']['source'] if len(json_data['query']['pages']) >0 else 'Not found'

wiki=wikipediaapi.Wikipedia('ko') # 한국어 세팅
wiki_dic = {} # Wikipedia페이지를 저장할 딕셔너리 -> 추후 중복데이터의 로딩시간을 절약
img_dic = {} # 이미지 링크를 저장할 딕셔너리 -> 동일

* 위키백과 API세팅

In [12]:
zoo_map = folium.Map(location=[user[0], user[1]], zoom_start = 14)
marker_dic = {'곤충' : 'yellow',
              '양서파충류' : 'blue',
              '저서무척추동물' : 'red',
              '조류' : '#990085', 
              '포유류' : 'green'}
tooltip = '나를 눌러줘!'

for i in want_list:
    # page dic 생성
    if df_want.iloc[i]['spcs_lcnm'] not in wiki_dic.values() :
        page_wiki = wiki.page(df_want.iloc[i]['spcs_lcnm'])
        wiki_dic[df_want.iloc[i]['spcs_lcnm']] = page_wiki
    else :
        page_wiki = wiki_dic[df_want.iloc[i]['spcs_lcnm']]
    
    # img dic 생성
    if page_wiki.exists() == True :
        if (page_wiki.title not in img_dic.values()) :
            wiki_img = get_wiki_main_image(page_wiki.title)
            img_dic[page_wiki.title] = wiki_img
        else : wiki_img[page_wiki.title]
    else :
        wiki_img = "http://www.visioncyber.kr/rtimages/n_sub/no_detail_img.gif"
    
    # 팝업창 요약설명 설정
    if page_wiki.exists() == True : summary = page_wiki.summary.split('.')[0]
    else : summary = df_want.iloc[i]['spcs_lcnm'] + '은(는) ' + df_want.iloc[i]['examin_realm_se_code_nm'] + '이다'
    
    # 팝업창 제목 추가
    popup = '<h3>나는 <strong>' + df_want.iloc[i]['spcs_lcnm'] + '</strong>야!</h3>'
    # 팝업창 이미지 추가
    popup += '<img src=' + wiki_img + ' class="Popup-img">'
    # 팝업창 요약설명 추가
    popup += '<p><h4>' + summary + '.</h4>'
    if page_wiki.exists() == True : popup += '<b><a href=' + page_wiki.canonicalurl + '>더 알아보기</a></b></p>'
    
    folium.CircleMarker(location=[df_want['kr_lati'][i], df_want['kr_long'][i]], popup=popup, 
                        tooltip=tooltip, radius=4, color=marker_dic[df_want.iloc[i]['examin_realm_se_code_nm']]).add_to(zoo_map)
zoo_map.save('map.html')
zoo_map